### Libraries Needed

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from scipy.optimize import brentq


# Extract local volatility

Extracting local volatility is crucial for modeling the dynamics of financial assets. Local volatility models allow the asset's volatility to vary with both price and time, providing a more accurate representation than constant volatility models.

To begin, we'll need to gather market data, including historical prices of options, spot prices, risk-free interest rates, and dividends. Using this data, we can construct the implied volatility surface, which displays implied volatility for various strike prices and maturities.

In [ ]:
def option_chains(ticker):
    """
    """
    asset = yf.Ticker(ticker)
    expirations = asset.options
    chains = pd.DataFrame()
    for expiration in expirations:
        opt = asset.option_chain(expiration)
        calls = opt.calls
        calls['optionType'] = "call"
        puts = opt.puts
        puts['optionType'] = "put"
        chain = pd.concat([calls, puts])
        chain['expiration'] = pd.to_datetime(expiration) + pd.DateOffset(hours=23, minutes=59, seconds=59)
        chains = pd.concat([chains, chain])
    chains["daysToExpiration"] = (chains.expiration - dt.datetime.today()).dt.days + 1
    most_common_date = chains['lastTradeDate'].mode()[0]
    chains_filtered = chains[chains['lastTradeDate'] == most_common_date]    
    return chains_filtered.reset_index(drop='True')

In [ ]:
def recup_strike(ticker,date):
    data = yf.download(ticker, start=date-timedelta(days=90), end=date)
    data['Returns'] = data['Adj Close'].pct_change().dropna()
    returns = data['Returns'].dropna()
    historical_volatility = np.std(data['Returns'].dropna()) * np.sqrt(252)    
    return data['Adj Close'].iloc[-1],historical_volatility

In [ ]:
ticker='AAPL'
options = option_chains(ticker)
date=options.loc[0,'lastTradeDate']
calls = options[options["optionType"] == "call"]
put = options[options["optionType"] == "put"]

In [ ]:
strike,vol=recup_strike(ticker,date)
print(f"The underlying price of {ticker} is {strike:.2f}$")
print(f"The initial volatility of {ticker} is {100*vol:.4f}%")

I noticed that on Yahoo Finance, implied volatilities are provided, but the specific method used to compute them isn't clear to me. To gain a deeper understanding of how implied volatilities are constructed, I've decided to compute them myself. This approach will allow me to delve into well-known financial mathematical formulas and better grasp their application.

To compute implied volatilities accurately, one typically needs to employ numerical methods. These methods aim to find the volatility value that, when input into the Black-Scholes formula, results in the market price of the option. A commonly used approach involves employing root-finding algorithms such as the Newton-Raphson method or Brent's method. I choose to use the Newton-Raphson method.

The Black-Scholes formula is foundational in options pricing, serving as a starting point for more sophisticated models that accommodate additional factors such as dividends, early exercise features, and varying interest rates. Its insights into risk-neutral valuation have significantly shaped modern finance.

The Black-Scholes formula Call and Put formula are :
$$
{\boxed{C(S_{0},K,r,T,\sigma )=S_{0}{\mathcal {N}}(d_{1})-K\mathrm {e} ^{-rT}{\mathcal {N}}(d_{2})}}
$$

$$
{\boxed{P(S_{0},K,r,T,\sigma )=-S_{0}{\mathcal {N}}(-d_{1})+K\mathrm {e} ^{-rT}{\mathcal {N}}(-d_{2})}}
$$

with : 
- $\mathcal {N}$ :  la fonction de répartition de la loi normale centrée réduite $\mathcal {N}{(0,1)}$
- $d1 = {\frac {1}{\sigma {\sqrt {T}}}}\left[\ln \left({\frac {S_{0}}{K}}\right)+\left(r+{\frac {1}{2}}\sigma ^{2}\right)T\right]$  
- $d2 = d1-\sigma {\sqrt {T}}$
- $S_0$ the current underlying price
- $K the strike option price
- $r$ the risk-free interest rate
- $T$ the time to maturity of the option
- $\sigma$ the volatility of the underlying asset


In [ ]:
def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
def bs_put_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return -S * norm.cdf(*d1) + K * np.exp(-r * T) * norm.cdf(-d2)

The Newton-Raphson method (also known as Newton's method) is an iterative root-finding algorithm that uses the derivative of a function to iteratively refine an initial guess for the root. It converges quickly for well-behaved functions but requires the derivative to be continuous and non-zero near the root.

The iterative formula for Newton-Raphson method is given by:

$x_{n+1} = x_{n} - \frac{f(x_{n})}{f'(x_{n})}$

where:
- $x_{n}$ is the current estimate of the root 
- $f'(x_{n})$ is the derivative of $f(x)$ with respect to $ x $.

In [ ]:
def newton_raphson(f, df, x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:
            return x
        dfx = df(x)
        if dfx == 0:
            break
        x = x - fx / dfx
    return x

Brent's method is another root-finding algorithm that combines the robustness of the bisection method with the speed of methods such as secant method, bisection method and inverse quadratic interpolation. It does not require the derivative of the function and generally converges faster than bisection.

Brent's method iteratively narrows down the interval containing the root by:
- <ins>Bisection method</ins> : Given an interval $[a, b]$ where $ f(a) $ and $ f(b) $ have opposite signs (i.e., $ f(a) \cdot f(b) < 0 $), the iterative formula for the bisection method is: $ \boxed{c = \frac{a + b}{2} }$
    - If $ f(c) = 0 $, then $ c $ is the root.
    - If $ f(a) \cdot f(c) < 0 $, then the root lies in $[a, c]$.
    - If $ f(b) \cdot f(c) < 0 $, then the root lies in $[c, b]$.

- <ins>Secant method</ins> : Given two initial guesses $ x_0 $ and $ x_1 $, the iterative formula for the secant method is: $\boxed{ x_{n+1} = x_n - \frac{f(x_n) \cdot (x_n - x_{n-1})}{f(x_n) - f(x_{n-1})} }$
    - $ x_n $ is the current estimate of the root,
    - $ f(x_n) $ and $ f(x_{n-1}) $ are the values of the function at $ x_n $ and $ x_{n-1} $, respectively.

- <ins>Inverse quadratic interpolation</ins> : Given three points $ x_0, x_1, x_2 $ where $ f(x_0), f(x_1), f(x_2) \neq 0 $, the formula for the new approximation $ x_3 $ is:

     $\boxed{ x_3 = x_0 - \frac{2 \left[ (x_0 - x_1)(f(x_0) - f(x_2)) - (x_0 - x_2)(f(x_0) - f(x_1)) \right]}{(x_0 - x_1)^2 (f(x_0) - f(x_2)) - (x_0 - x_2)^2 (f(x_0) - f(x_1))}}$

    - $ x_3 $ is the new approximation of the root,
    - $ x_0, x_1, x_2 $ are three successive points used for interpolation,
    - $ f(x_0), f(x_1), f(x_2) $ are the values of the function evaluated at $ x_0, x_1, x_2 $, respectively.



In [ ]:
def implied_vol_objective(sigma, market_price, S, K, T, r):
    return bs_call_price(S, K, T, r, sigma) - market_price

def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)


Now that we got the numerical methods and the objective function (BS result - Yahoo Finance prices), we can compute these formula to have implied volatilities.

In [ ]:
calls['strike'].values

In [106]:
strikes = calls['strike'].values
market_prices = calls['lastPrice'].values
S = strike
times_to_maturity = (calls['daysToExpiration'].values) / 365
r = 0.05048 #1 Year Treasury USBond Rate 

implied_vols_newton = []
for K, price, T in zip(strikes, market_prices, times_to_maturity):
    f = lambda sigma: bs_call_price(S, K, T, r, sigma) - price
    df = lambda sigma: vega(S, K, T, r, sigma)
    initial_guess = 0.2
    implied_vol = newton_raphson(f, df, initial_guess)
    implied_vols_newton.append(implied_vol)

# Print the results
print("Implied Volatilities using Brent's Method:")
for K, iv in zip(strikes, implied_vols_brentq):
    print(f"Strike Price: {K}, Implied Volatility: {iv:.4f}")

print("\nImplied Volatilities using Newton-Raphson Method:")
for K, iv in zip(strikes, implied_vols_newton):
    print(f"Strike Price: {K}, Implied Volatility: {iv:.4f}")

Implied Volatilities using Brent's Method:

Implied Volatilities using Newton-Raphson Method:
Strike Price: 212.5, Implied Volatility: -3129.7557
Strike Price: 180.0, Implied Volatility: 895.3673
Strike Price: 207.5, Implied Volatility: 0.2170
Strike Price: 225.0, Implied Volatility: 0.2454
Strike Price: 245.0, Implied Volatility: 0.2470
Strike Price: 225.0, Implied Volatility: 0.2232
Strike Price: 230.0, Implied Volatility: 0.2239
Strike Price: 180.0, Implied Volatility: 0.2700
Strike Price: 215.0, Implied Volatility: 0.2243
Strike Price: 230.0, Implied Volatility: 0.2200
Strike Price: 215.0, Implied Volatility: 0.2328
Strike Price: 300.0, Implied Volatility: 0.2345


In [107]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expiration,daysToExpiration
0,AAPL240705C00212500,2024-07-03 16:59:32+00:00,212.5,9.10,8.80,9.50,1.000000,12.345678,1400.0,13370,0.478033,True,REGULAR,USD,call,2024-07-05 23:59:59,2
2,AAPL240719C00180000,2024-07-03 16:59:32+00:00,180.0,42.00,41.80,42.15,1.200001,2.941178,160.0,34131,0.578129,True,REGULAR,USD,call,2024-07-19 23:59:59,16
3,AAPL240719C00207500,2024-07-03 16:59:32+00:00,207.5,14.80,14.65,15.05,1.060000,7.714704,189.0,0,0.296760,True,REGULAR,USD,call,2024-07-19 23:59:59,16
7,AAPL240809C00225000,2024-07-03 16:59:32+00:00,225.0,5.85,5.55,5.95,0.410000,7.536761,167.0,269,0.266243,False,REGULAR,USD,call,2024-08-09 23:59:59,37
8,AAPL240816C00245000,2024-07-03 16:59:32+00:00,245.0,1.36,1.33,1.39,0.070000,5.426361,770.0,0,0.259285,False,REGULAR,USD,call,2024-08-16 23:59:59,44
10,AAPL240920C00225000,2024-07-03 16:59:32+00:00,225.0,8.70,8.70,8.80,0.500000,6.097561,2071.0,0,0.251900,False,REGULAR,USD,call,2024-09-20 23:59:59,79
11,AAPL240920C00230000,2024-07-03 16:59:32+00:00,230.0,6.65,6.60,6.70,0.400000,6.400002,2776.0,0,0.247871,False,REGULAR,USD,call,2024-09-20 23:59:59,79
12,AAPL241018C00180000,2024-07-03 16:59:32+00:00,180.0,45.00,44.75,45.20,1.650001,3.806232,175.0,0,0.391302,True,REGULAR,USD,call,2024-10-18 23:59:59,107
13,AAPL241018C00215000,2024-07-03 16:59:32+00:00,215.0,16.05,15.95,16.20,0.769999,5.039264,114.0,10275,0.268623,True,REGULAR,USD,call,2024-10-18 23:59:59,107
14,AAPL241018C00230000,2024-07-03 16:59:32+00:00,230.0,8.30,8.20,8.35,0.520000,6.683804,613.0,0,0.248451,False,REGULAR,USD,call,2024-10-18 23:59:59,107


In [ ]:
# print the expirations
set(calls.expiration)

# select an expiration to plot
calls_at_expiry = calls[calls["expiration"] == "2024-07-19 23:59:59"]

# filter out low vols
filtered_calls_at_expiry = calls_at_expiry[calls_at_expiry.impliedVolatility >= 0.001]

# set the strike as the index so pandas plots nicely
filtered_calls_at_expiry[["strike", "impliedVolatility"]].set_index("strike").plot(
    title="Implied Volatility Skew", figsize=(7, 4)
)